In [ ]:
import pandas as pd

import os
import sys
from dotenv import load_dotenv

load_dotenv()

REPO_PATH =  os.getenv('REPO_PATH')

sys.path.insert(0, rf'{REPO_PATH}src_HF')
from utils import *

### Merge headlines with stories

In [ ]:
TOPIC = 'CWP'

text_df = pd.read_csv(rf'{REPO_PATH}data\raw_news_headlines\EIKON_{TOPIC}_NEWS.csv')
story_dict = load_json(rf'{REPO_PATH}data\raw_news_stories\EIKON_{TOPIC}_NEWS_FULL.json')

# add fullStory column to text_df
text_df['fullStory'] = text_df['storyId'].map(story_dict)

# remove all rows where fullStory is NaN
text_df.dropna(subset=['fullStory']); print(text_df.shape[0])
# remove all rows where fullStory is 'error'
text_df = text_df[text_df['fullStory'] != 'error']; print(text_df.shape[0])
# remove all rows where fullStory is float
text_df = text_df[text_df['fullStory'].apply(lambda x: isinstance(x, str))]; print(text_df.shape[0])

# sort df by date
text_df['date'] = pd.to_datetime(text_df['date'])
text_df.sort_values(by='date', inplace=True)

# reset index
text_df.reset_index(drop=True, inplace=True)

display(text_df)

### Check for duplicates

In [ ]:
# check how many fullStory duplicate rows
print('Removed ' + str(text_df['fullStory'].duplicated().sum()) + ' duplicate stories')

# remove all fullStory duplicate rows
text_df.drop_duplicates(subset=['fullStory'], inplace=True)

# reset index
text_df.reset_index(drop=True, inplace=True)

### Save data

In [ ]:
text_df.to_json(
    rf'{REPO_PATH}data\news_data\EIKON_{TOPIC}_NEWS_COMPLETE.json',
    orient='records',
    lines=True
)

print('Saved ' + str(text_df.shape[0]) + ' unique stories to json')